In [1]:
import pickle
import pandas as pd
import torch
from pykeen import predict
from pykeen.pipeline import pipeline
from glob import glob
from pykeen.datasets import OpenBioLink
from Bio.Entrez.Parser import CorruptedXMLError  


/nfs_home/users/mbvk/.conda/envs/GNN_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

We have trained 10 models with the best hyperparamters we will load them into a list

In [3]:
model0 = torch.load('replicates/replicate-00000/trained_model.pkl', map_location=torch.device('cpu'))
model1 = torch.load('replicates/replicate-00001/trained_model.pkl', map_location=torch.device('cpu'))
model2 = torch.load('replicates/replicate-00002/trained_model.pkl', map_location=torch.device('cpu'))
model3 = torch.load('replicates/replicate-00003/trained_model.pkl', map_location=torch.device('cpu'))
model4 = torch.load('replicates/replicate-00004/trained_model.pkl', map_location=torch.device('cpu'))
model5 = torch.load('replicates/replicate-00005/trained_model.pkl', map_location=torch.device('cpu'))
model6 = torch.load('replicates/replicate-00006/trained_model.pkl', map_location=torch.device('cpu'))
model7 = torch.load('replicates/replicate-00007/trained_model.pkl', map_location=torch.device('cpu'))
model8 = torch.load('replicates/replicate-00008/trained_model.pkl', map_location=torch.device('cpu'))
model9 = torch.load('replicates/replicate-00009/trained_model.pkl', map_location=torch.device('cpu'))

In [4]:
models = [model0,
         model1,
         model2,
         model3,
         model4,
         model5,
         model6,
         model7,
         model8,
         model9]

In [5]:
training = OpenBioLink().training
testing = OpenBioLink().testing

You're trying to map triples with 2052 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 2047 from 183011 triples were filtered out
You're trying to map triples with 2052 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 2047 from 183011 triples were filtered out


Loads the Genes we know are IR related from lis file

In [6]:
genes_important = pd.read_csv('../IR_GeneID_names.csv')

In [7]:
genes_important['Gene ID'] = genes_important['Gene ID'].dropna().astype(int).astype(str)
known_values = genes_important['Gene ID']

In [8]:
known_values

0         19
1       5243
2       8647
3       9429
4      64240
       ...  
619     4904
620     7534
621    84327
622    60685
623    54469
Name: Gene ID, Length: 624, dtype: object

# Maps names id to name

In [21]:
from Bio import Entrez  
  
def get_gene_names(gene_ids):
    Entrez.email = 'MB00_new@gmail.com'  # Provide your email address to NCBI

    # Fetch the gene summaries from NCBI
    handle = Entrez.efetch(db='gene',  retmode='xml',id=gene_ids, rettype='gene')
    #records = Entrez.read(handle)
    # Parse the XML data using Entrez.read(), with error handling  
    try:  
        records = Entrez.read(handle)  
    except CorruptedXMLError as e:  
        print(f"Error: {e}")  
        records = None  

    gene_map = {}
    for record in records:
        gene_id = record['Entrezgene_track-info']['Gene-track']['Gene-track_geneid']
        gene_name = record['Entrezgene_gene']['Gene-ref']['Gene-ref_locus']
        gene_map[gene_id] = gene_name

    return gene_map

In [12]:
genes_important

,Input ID,Gene ID
0,ABCA1,19
1,ABCB1,5243
2,ABCB11,8647
3,ABCG2,9429
4,ABCG5,64240
...,...,...
619,YBX1,4904
620,YWHAZ,7534
621,ZBED3,84327
622,ZFAND3,60685


# Make predictions

The predictions made here filter out the edges present in the training dataset

In [13]:
genes = []
#genes that match the list provided by Lis
true_genes = []
for model in models:
    results = predict.predict_target(model=model, tail="HP:0000855", relation="GENE_PHENOTYPE", triples_factory=testing).filter_triples(training).df
    #selects the 100 most likley genes to have edge with IR node
    res = results[:100]['head_label'].str.replace('NCBIGENE:', '')
    
    match_genes = res[res.isin(known_values).to_list()]
    
    '''gene_name_map = get_gene_names(list(res))'''
    genes.append(res)
    true_genes.append(match_genes)
    

Matches the genes numbers to the ID

In [15]:
names = []
for i in range(len(genes)):
    names.append(get_gene_names(genes[i]))

In [22]:
match_names = []
for i in range(len(genes)):
    res = genes[i]
    match_genes = res[res.isin(known_values).to_list()]
    match_names.append(get_gene_names(match_genes))

Average number of correctly predicted gene ID's

In [35]:
list_of_genes = [[value for value in dictionary.values()] for dictionary in genes[:]]
df = pd.DataFrame(zip(*list_of_genes), columns=[f"model_{i}" for i in range(len(list_of_genes))])


In [29]:
df.to_csv('RotatE_10_prediction_models.csv')